In [1]:
import sys
import skvideo.io
import skvideo.datasets

/home/misha/tensorflow/lib/python3.5/site-packages/skvideo/__init__.py:356: UserWarning: avconv/avprobe not found in path: 
  warnings.warn("avconv/avprobe not found in path: " + str(path), UserWarning)


In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution3D, MaxPooling3D

Using TensorFlow backend.


In [3]:
from keras.optimizers import SGD, RMSprop, Nadam, Adam
from keras.utils import np_utils, generic_utils

In [4]:
import tensorflow as tf
import os
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import cv2
from sklearn.cross_validation import train_test_split
from sklearn import cross_validation
from sklearn import preprocessing

/home/misha/tensorflow/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [5]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [6]:
hello = tf.constant('Hello, TensorFlow!')
sess = tf.Session()
print(sess.run(hello))

b'Hello, TensorFlow!'


### Image specification + Random

In [7]:
img_rows,img_cols,img_depth=120,160,15
r_state = 42
np.random.seed(r_state)
tf.set_random_seed(r_state)

### Training data

In [8]:
train_nb, test_nb = 196, 100
x_train = np.zeros((0,img_rows,img_cols,img_depth)).astype('float32')# variable to store entire dataset
x_test = np.zeros((0,img_rows,img_cols,img_depth)).astype('float32')
# fr_arr = fr_arr.astype('float32')
print(x_train.shape, x_test.shape)

(0, 120, 160, 15) (0, 120, 160, 15)


In [ ]:
print(x_test)

In [9]:
def readFrames(vdata,start,end,width,step): # width of the window and step
    x_arr = np.zeros((0,img_rows,img_cols,img_depth))
    
    frame_number = 0
    uniq_frames_nb = end - start  # + width - (end - start) % width
    uniq_frames_nb = uniq_frames_nb - (uniq_frames_nb - width) % step
    print('uniq frames: ', uniq_frames_nb,start,end)
    frames = []
    
    for frame in vdata:
#         print('fr_nb:', frame_number,'out of:', start + uniq_frames_nb)
        if (frame_number >= start and frame_number < start + uniq_frames_nb):
#             print('i\'m inside')
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            res_img = cv2.resize(frame, (img_cols, img_rows), interpolation=cv2.INTER_AREA)
            frames.append(res_img)
        if frame_number > start + uniq_frames_nb:
            frame_number = 0
            break
        frame_number += 1
    print('real frames: ', len(frames))
        
    # sliding window itself
    nb_x = (uniq_frames_nb - width) / step + 1
    print('adding',nb_x,'windows')
    for i in range(int(nb_x)):
        fr_arr = np.zeros((img_depth,img_rows,img_cols))
        fr_arr = fr_arr.astype('float32')
#         print(fr_arr.shape, img_depth, len(frames), ((int(nb_x) - 1) * step + img_depth - 1))
        
        for j in range(img_depth):
            fr_arr[j] = frames[i * step + j]
        fr_arr = np.rollaxis(fr_arr,0,3)
        fr_arr = np.expand_dims(fr_arr, axis = 0)
        x_arr = np.concatenate((x_arr, fr_arr), axis=0) 
        
    return x_arr

In [10]:
def loadFrames(fr_arr, v_name, v_ext, acc, border=1000000, start_bias=0):
    fr_numbers = open(v_name + '.csv', 'r')
    st_len = fr_arr.shape[0]
    
    for line in fr_numbers:
        line = line.split(',')
        print(line)
        videodata = skvideo.io.vreader(v_name + v_ext)
        if int(line[2]) == acc: # int(line[1])
            print('class: ', acc)
            fr_arr = np.concatenate((fr_arr,readFrames(videodata,int(line[0]) + start_bias,int(line[1]),img_depth,5)), axis = 0)
        if fr_arr.shape[0] > border:
            break
    
    print('done:', fr_arr.shape)
    return fr_arr

In [ ]:
print(x_test.shape)

#### Load YouTube videos

In [ ]:
print(x_test)

In [ ]:
x_test = loadFrames(x_test, '/home/misha/Documents/Thesis/Dataset/BNG_white_1', '.mov', 1,start_bias=-20)
# print(x)

In [ ]:
x_test = loadFrames(x_test, '/home/misha/Documents/Thesis/Dataset/BNG_white_1', '.mov', 0, border=158)

In [ ]:
x_test = x_test[0:158]
nb_classes = 2

In [ ]:
y_test = np.zeros(158)
y_test[0:79] = 1
y_test[79:] = 0
print(y_test.shape)
y_test = np_utils.to_categorical(y_test, nb_classes)
print(y_test.shape)

In [11]:
filename = 'small_dataset/0-0'
x_train = loadFrames(x_train, filename, '.mp4', 0)

print(x_train.shape)

['0', '577', '0\n']
class:  0
uniq frames:  575 0 577
real frames:  575
adding 113.0 windows
done: (113, 120, 160, 15)
(113, 120, 160, 15)


In [12]:
filename = 'small_dataset/0 4K camera example for Traffic Monitoring (Road, Interchage) [360p]'
x_train = loadFrames(x_train, filename, '.mp4', 0)

print(x_train.shape)

['450', '1250', '0\n']
class:  0
uniq frames:  800 450 1250
real frames:  800
adding 158.0 windows
['1290', '1983', '0\n']
class:  0
uniq frames:  690 1290 1983
real frames:  690
adding 136.0 windows
['2350', '2536', '0\n']
class:  0
uniq frames:  185 2350 2536
real frames:  185
adding 35.0 windows
['2570', '3040', '0\n']
class:  0
uniq frames:  470 2570 3040
real frames:  470
adding 92.0 windows
['3075', '3640', '0\n']
class:  0
uniq frames:  565 3075 3640
real frames:  565
adding 111.0 windows
done: (645, 120, 160, 15)
(645, 120, 160, 15)


In [13]:
filename = 'small_dataset/0 36th St & Wetmore Ave - Double Stop Sign Run In Front Of Cross Traffic 2017-02-27 [360p]'
x_train = loadFrames(x_train, filename, '.mp4', 0)

print(x_train.shape)

['0', '740', '0\n']
class:  0
uniq frames:  740 0 740
real frames:  740
adding 146.0 windows
done: (791, 120, 160, 15)
(791, 120, 160, 15)


In [14]:
filename = 'small_dataset/0 36th St & Wetmore Ave - Everett PD Vehicle Nearly Rolls Through Stop 2017-02-27 [360p]'
x_train = loadFrames(x_train, filename, '.mp4', 0)

print(x_train.shape)

['0', '400', '0\n']
class:  0
uniq frames:  400 0 400
real frames:  400
adding 78.0 windows
done: (869, 120, 160, 15)
(869, 120, 160, 15)


In [15]:
filename = 'small_dataset/0 Full HD Car megapixel CCTV camera Video Demo [360p]'
x_train = loadFrames(x_train, filename, '.mp4', 0)

print(x_train.shape)

['0', '600', '0\n']
class:  0
uniq frames:  600 0 600
real frames:  600
adding 118.0 windows
done: (987, 120, 160, 15)
(987, 120, 160, 15)


### Class 1

In [16]:
filename = 'small_dataset/1 BeamNG.drive - CCTV Crash Compilation #3 [360p]'
x_train_temp = np.zeros((0,img_rows,img_cols,img_depth))
x_train_temp = loadFrames(x_train_temp, filename, '.mp4', 1, border=x_train.shape[0])
x_train = np.concatenate((x_train, x_train_temp), axis=0) 

print(x_train.shape)

['250', '328', '1\n']
class:  1
uniq frames:  75 250 328
real frames:  75
adding 13.0 windows
['329', '495', '1\n']
class:  1
uniq frames:  165 329 495
real frames:  165
adding 31.0 windows
['560', '728', '1\n']
class:  1
uniq frames:  165 560 728
real frames:  165
adding 31.0 windows
['840', '1020', '1\n']
class:  1
uniq frames:  180 840 1020
real frames:  180
adding 34.0 windows
['1250', '1423', '1\n']
class:  1
uniq frames:  170 1250 1423
real frames:  170
adding 32.0 windows
['1485', '1730', '1\n']
class:  1
uniq frames:  245 1485 1730
real frames:  245
adding 47.0 windows
['1835', '2280', '1\n']
class:  1
uniq frames:  445 1835 2280
real frames:  445
adding 87.0 windows
['2505', '2685', '1\n']
class:  1
uniq frames:  180 2505 2685
real frames:  180
adding 34.0 windows
['2740', '2970', '1\n']
class:  1
uniq frames:  230 2740 2970
real frames:  230
adding 44.0 windows
['2990', '3240', '1\n']
class:  1
uniq frames:  250 2990 3240
real frames:  250
adding 48.0 windows
['3300', '3600',

In [17]:
filename = 'small_dataset/1 Highest Quality CCTV Crashes On The Web [360p]'

x_train_temp = np.zeros((0,img_rows,img_cols,img_depth))
x_train_temp = loadFrames(x_train_temp, filename, '.mp4', 1, border=200)
x_train = np.concatenate((x_train, x_train_temp), axis=0) 

print(x_train.shape)

['155', '365', '1\n']
class:  1
uniq frames:  210 155 365
real frames:  210
adding 40.0 windows
['480', '625', '1\n']
class:  1
uniq frames:  145 480 625
real frames:  145
adding 27.0 windows
['690', '850', '1\n']
class:  1
uniq frames:  160 690 850
real frames:  160
adding 30.0 windows
['902', '1030', '1\n']
class:  1
uniq frames:  125 902 1030
real frames:  125
adding 23.0 windows
['1225', '1425', '1\n']
class:  1
uniq frames:  200 1225 1425
real frames:  200
adding 38.0 windows
['1615', '1850', '1\n']
class:  1
uniq frames:  235 1615 1850
real frames:  235
adding 45.0 windows
done: (203, 120, 160, 15)
(1998, 120, 160, 15)


In [18]:
print(x_test.shape) # 987, 2422 = 1974

(0, 120, 160, 15)


In [19]:
x_train = x_train[0:1974]
print(x_train.shape)

(1974, 120, 160, 15)


In [ ]:
vid_to_viz = x_train[50]
# vid_to_viz = np.flip(np.squeeze(X_val_new[5], axis=3),1)
vid_to_viz = np.rollaxis(vid_to_viz, 2, 0)
_, axes = plt.subplots(ncols=5,nrows=3,figsize=(15,7))
for i in range(3):
    for j in range(5):
        axes[i][j].axis('off')
        axes[i][j].set_title(i * 5 + j)
        axes[i][j].imshow(vid_to_viz[i * 5 + j],cmap = 'gray')
                   
plt.show()

In [ ]:
# vid_to_viz = x_train_0[3]
# vid_to_viz = np.flip(np.squeeze(X_val_new[5], axis=3),1)
# vid_to_viz = np.rollaxis(vid_to_viz, 2, 0)
# _, axes = plt.subplots(ncols=5,nrows=3,figsize=(15,7))
# for i in range(3):
#     for j in range(5):
#         axes[i][j].axis('off')
#         axes[i][j].set_title(i * 5 + j)
#         axes[i][j].imshow(vid_to_viz[i * 5 + j],cmap = 'gray')
                   
# plt.show()

### Open files

In [ ]:
# y_test = np_utils.to_categorical(y_test, nb_classes)
print(x_test.shape)

In [20]:
x_test = np.expand_dims(x_test, axis=4)
print(x_test.shape)

(0, 120, 160, 15, 1)


In [ ]:
from sklearn.utils import shuffle
# x_test, y_test = shuffle(x_test,y_test, random_state = r_state)

### Reading accident class training

##### Image resize

### Reading all frames classes (takes long time)

### CNN Training parameters

In [ ]:
print(x_train.shape)

In [21]:
x_train = np.expand_dims(x_train, axis=4)
print(x_train.shape)

(1974, 120, 160, 15, 1)


##### Assign label to each class

In [22]:
y_train = np.ones((x_train.shape[0],),dtype=int)
y_train[0:987] = 0
y_train[987:] = 1
print(y_train.shape)

(1974,)


##### CNN training parameters

In [23]:
batch_size = 3
nb_classes = 2 
nb_epoch = 5


##### convert class vectors to binary class matrices

In [24]:
y_train = np_utils.to_categorical(y_train, nb_classes)
print(y_train.shape)

(1974, 2)


In [ ]:
# shuffle train data
# x_train, y_train = shuffle(x_train,y_train, random_state = r_state)

x_train, y_train = shuffle(x_train,y_train, random_state = r_state)

# x_train = np.expand_dims(x_train, axis=4)
print(x_train.shape)

In [ ]:
x_test, y_test = shuffle(x_test, y_test, random_state = r_state)
print(x_test.shape)

##### number of convolutional filters to use at each layer

In [26]:
nb_filters = [32, 32]

##### level of pooling to perform at each layer (POOL x POOL)

In [27]:
nb_pool = [2, 2]

##### Level of convolution to perform at each layer (CONV x CONV)

In [28]:
nb_conv = [3,3]

### Pre-processing

In [29]:
x_train = x_train.astype('float32')
# train_set -= np.mean(train_set)
x_train /=np.max(x_train)

In [ ]:
print(img_rows, img_cols, img_depth)

In [30]:
x_test = x_test.astype('float32')
x_test /= np.max(x_test)

### Define model

In [31]:
model = Sequential()
model.add(Convolution3D(16, (4, 4, 1),
input_shape=(img_rows, img_cols, img_depth, 1), activation='relu'))

# model.add(Convolution3D(16, (3, 3, 2), activation='relu'))

model.add(MaxPooling3D(pool_size=(nb_pool[0], nb_pool[0], nb_pool[0])))
model.add(Dropout(0.2))

model.add(Convolution3D(32, (4, 4, 4), activation='relu'))

model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.2))

model.add(Convolution3D(64, (3, 3, 2), activation='relu'))
model.add(Dropout(0.2))

# model.add(Convolution3D(128, (3, 3, 1), activation='relu'))

model.add(MaxPooling3D(pool_size=(2, 2, 1)))

model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(128, init='normal', activation='relu'))
model.add(Dense(256, init='normal', activation='relu'))

model.add(Dropout(0.2))

model.add(Dense(nb_classes,init='normal'))

model.add(Activation('softmax'))

# model.optimizer.lr.assign(0.1)

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=Nadam(lr=0.0005))


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:26: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, activation="relu", kernel_initializer="normal")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:27: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(256, activation="relu", kernel_initializer="normal")`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:31: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(2, kernel_initializer="normal")`


In [32]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 117, 157, 15, 16)  272       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 58, 78, 7, 16)     0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 58, 78, 7, 16)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 55, 75, 4, 32)     32800     
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 27, 37, 2, 32)     0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 27, 37, 2, 32)     0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 25, 35, 1, 64)     36928     
__________

In [ ]:
x_train = np.flip(x_train, 2)

In [33]:
print(x_train.shape, x_test.shape)

(1579, 120, 160, 15, 1) (395, 120, 160, 15, 1)


In [ ]:
x_test = np.flip(x_test, 2)

In [25]:
# Split the data
x_train, x_test, y_train, y_test =  train_test_split(x_train, y_train, test_size=0.2, random_state=r_state)

In [ ]:
# X_train_new = np.expand_dims(X_train_new, axis=4)
# print(X_train_new.shape)

# X_val_new = np.expand_dims(X_val_new, axis=4)
# print(X_val_new.shape)

#### Training

In [36]:
batch_size = 25
nb_epoch = 2

In [37]:
hist = model.fit(x_train, y_train, validation_data=(x_test,y_test),
          batch_size=batch_size, epochs=nb_epoch, shuffle=True)

Train on 1579 samples, validate on 395 samples
Epoch 1/2
1579/1579 [==============================] - 17s - loss: 0.4166 - acc: 0.8018 - val_loss: 0.1162 - val_acc: 0.9949
Epoch 2/2
1579/1579 [==============================] - 15s - loss: 0.0540 - acc: 0.9867 - val_loss: 0.0228 - val_acc: 0.9975


### Split the data

### Train the model

### Evaluate the model

In [ ]:
X_val_new = np.flip(X_val_new, 2)

In [ ]:
X_val_new = np.

In [ ]:
score = model.evaluate(x_test, y_test, batch_size=batch_size)
print('Test score:', score[0])
print('Test accuracy:', score[1]) 


In [ ]:
train_loss=hist.history['loss']
val_loss=hist.history['val_loss']
train_acc=hist.history['acc']
val_acc=hist.history['val_acc']

plt.plot(train_loss)
plt.plot(val_loss)
plt.plot(train_acc)
plt.plot(val_acc)
plt.xlabel('num of Epochs')
plt.grid(True)
plt.legend(['train_loss','val_loss','train_acc', 'val_acc'])

plt.show()

### Plots the result

In [ ]:
# train_loss=hist.history['loss']
# val_loss=hist.history['val_loss']
# train_acc=hist.history['acc']
# val_acc=hist.history['val_acc']
# xc=range(10)

# plt.figure(1,figsize=(7,5))
# plt.plot(xc,train_loss)
# plt.plot(xc,val_loss)
# plt.xlabel('num of Epochs')
# plt.ylabel('loss')
# plt.title('train_loss vs val_loss')
# plt.grid(True)
# plt.legend(['train','val'])
# print(plt.style.available) # use bmh, classic,ggplot for big pictures
# plt.style.use(['classic'])

# plt.figure(2,figsize=(7,5))
# plt.plot(xc,train_acc)
# plt.plot(xc,val_acc)
# plt.xlabel('num of Epochs')
# plt.ylabel('accuracy')
# plt.title('train_acc vs val_acc')
# plt.grid(True)
# plt.legend(['train','val'],loc=4)
# #print plt.style.available # use bmh, classic,ggplot for big pictures
# plt.style.use(['classic'])